In [22]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.log.util import sorting
from scipy.stats import wasserstein_distance
from sklearn import preprocessing

from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance

import warnings
warnings.filterwarnings("ignore")

In [23]:
import re
def extract_first_float(cell):
    if isinstance(cell, str):
        # Use regular expression to extract the first float and the value in brackets
        match = re.match(r'(\d+\.\d+)(?: \((\d+\.\d+)\))?', cell)
        if match:
            return float(match.group(1)), (match.group(2)) if match.group(2) else ''
        else:
            return float('inf'), ''
    else:
        return cell, ''

In [24]:
def highlight_min_max(s):
    """
    Highlight the minimum value in green and the maximum value in red for each column.
    """
    is_min = s == s.min()
    is_max = s == s.max()
    min_max_style = ['background-color: green' if v else '' for v in is_min]
    for i, v in enumerate(is_max):
        if v:
            min_max_style[i] = 'background-color: red'
    return min_max_style

In [25]:
def align_column_names(df):
    if 'case:concept:name' in df.columns:
        df = df.rename(columns={'case:concept:name': 'case_id'})
    elif 'caseid' in df.columns:
        df = df.rename(columns={'caseid': 'case_id'})
    if 'Activity' in df.columns:
        df = df.rename(columns={'Activity': 'activity'})
    elif 'activity_name' in df.columns:
        df = df.rename(columns={'activity_name': 'activity'})
    elif 'task' in df.columns:
        df = df.rename(columns={'task': 'activity'})
    elif 'concept:name' in df.columns:
        df = df.rename(columns={'concept:name': 'activity'})
    if 'Resource' in df.columns:
        df = df.rename(columns={'Resource': 'resource'})
    elif 'user' in df.columns:
        df = df.rename(columns={'user': 'resource'})
    elif 'agent' in df.columns:
        if 'resource' in df.columns:
            df = df.drop(['resource'], axis=1)
        df = df.rename(columns={'agent': 'resource'})
    elif 'org:resource' in df.columns:
        df = df.rename(columns={'org:resource': 'resource'})
    if 'start_timestamp' in df.columns:
        df = df.rename(columns={'start_timestamp': 'start_time'})
    if 'end_timestamp' in df.columns:
        df = df.rename(columns={'end_timestamp': 'end_time'})
    # for SIMOD simulated logs
    if 'start_time' in df.columns:
        df = df.rename(columns={'start_time': 'start_time'})
    if 'end_time' in df.columns:
        df = df.rename(columns={'end_time': 'end_time'})
    if 'start:timestamp' in df.columns:
        df = df.rename(columns={'start:timestamp': 'start_time'})
    if 'time:timestamp' in df.columns:
        df = df.rename(columns={'time:timestamp': 'end_time'})
    return df

In [26]:
def main_(log_paths, name_experiments):
    def perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log):
        for i in range(10):
            # print(f"Evaluate simulation {i}")
            path_simulated_file = PATH_SIMULATED_LOG + '/simulated_log_' + str(i) + '.csv'
            # read simulated log and align column names
            simulated_log = pd.read_csv(path_simulated_file)
            simulated_log = align_column_names(simulated_log)
            # print(simulated_log)
            # print("########")
            # print(simulated_log[event_log_ids.activity].unique())
            simulated_log[event_log_ids.start_time] = pd.to_datetime(simulated_log[event_log_ids.start_time], utc=True, format='mixed')
            simulated_log[event_log_ids.end_time] = pd.to_datetime(simulated_log[event_log_ids.end_time], utc=True, format='mixed')

            # Call passing the event logs, and its column ID mappings
            ngd = n_gram_distribution_distance(test_log, event_log_ids, simulated_log, event_log_ids, n=3)
            all_metrics['NGD'].append(ngd)

            # Call passing the event logs, its column ID mappings, timestamp type, and discretize function
            aedd = absolute_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
            )
            all_metrics['AEDD'].append(aedd)

            # cadd = case_arrival_distribution_distance(
            #     test_log, event_log_ids,  # First event log and its column id mappings
            #     simulated_log, event_log_ids,  # Second event log and its column id mappings
            #     discretize_event=discretize_to_hour  # Function to discretize each timestamp (default by hour)
            # )
            # all_metrics['CADD'].append(cadd)

            cedd = circadian_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
            )
            all_metrics['CEDD'].append(cedd)

            redd = relative_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
            )
            all_metrics['REDD'].append(redd)


            ctdd = cycle_time_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                bin_size=pd.Timedelta(hours=1)  # Bins of 1 hour
            )
            all_metrics['CTDD'].append(ctdd)

        return all_metrics
    
    number_evaluations = len(log_paths)

    # Set event log column ID mapping
    event_log_ids = EventLogIDs(  # These values are stored in DEFAULT_CSV_IDS
        case="case_id",
        activity="activity",
        start_time="start_time",
        end_time="end_time",
        resource='resource'
    )

    index_names = name_experiments
    results_df = pd.DataFrame(index=index_names)
    mean_results = pd.DataFrame(index=index_names)

    for experiment in range(number_evaluations):
        # Read and transform time attributes
        test_log = pd.read_csv(log_paths[experiment][0])
        test_log = align_column_names(test_log)
        test_log[event_log_ids.start_time] = pd.to_datetime(test_log[event_log_ids.start_time], utc=True, format='mixed')
        test_log[event_log_ids.end_time] = pd.to_datetime(test_log[event_log_ids.end_time], utc=True, format='mixed')

        PATH_SIMULATED_LOG = log_paths[experiment][1]

        all_metrics = {
            'NGD': [],
            'AEDD': [],
            # 'CADD': [],
            'CEDD': [],
            'REDD': [],
            'CTDD': [],
        }

        all_metrics = perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log)

        mean_results.loc[index_names[experiment], 'N-Gram Distribution Distance'] = round(np.mean(all_metrics['NGD']), 3)
        mean_results.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = round(np.mean(all_metrics['AEDD']), 3)
        # mean_results.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = round(np.mean(all_metrics['CADD']), 3)
        mean_results.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = round(np.mean(all_metrics['CEDD']), 3)
        mean_results.loc[index_names[experiment], 'Relative Event Distribution Distance'] = round(np.mean(all_metrics['REDD']), 3)
        mean_results.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = round(np.mean(all_metrics['CTDD']), 3)


    

        results_df.loc[index_names[experiment], 'N-Gram Distribution Distance'] = f"{round(np.mean(all_metrics['NGD']), 3)} ({round(np.std(all_metrics['NGD']), 3)})"
        results_df.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = f"{round(np.mean(all_metrics['AEDD']), 3)} ({round(np.std(all_metrics['AEDD']), 3)})"
        # results_df.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = f"{round(np.mean(all_metrics['CADD']), 3)} ({round(np.std(all_metrics['CADD']), 3)})"
        results_df.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = f"{round(np.mean(all_metrics['CEDD']), 3)} ({round(np.std(all_metrics['CEDD']), 3)})"
        results_df.loc[index_names[experiment], 'Relative Event Distribution Distance'] = f"{round(np.mean(all_metrics['REDD']), 3)} ({round(np.std(all_metrics['REDD']), 3)})"
        results_df.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = f"{round(np.mean(all_metrics['CTDD']), 3)} ({round(np.std(all_metrics['CTDD']), 3)})"

    return mean_results, results_df


### Loan Application

In [27]:
PATH_TEST_LOG_MAS = '../simulated_data/LoanApp.csv/orchestrated_IM/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/LoanApp.csv/orchestrated_IM'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
]

name_experiments = ['IM']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
IM,0.087 (0.01),3.565 (1.038),0.23 (0.04),2.89 (1.031),3.096 (1.47)


In [28]:
PATH_TEST_LOG_MAS = '../simulated_data/P2P/orchestrated_IM/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/P2P/orchestrated_IM'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
]

name_experiments = ['IM']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
IM,0.537 (0.02),1118.416 (14.504),1.597 (0.163),661.06 (12.76),490.121 (14.374)


In [29]:
PATH_TEST_LOG_MAS = '../simulated_data/cvs_pharmacy/orchestrated_IM/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/cvs_pharmacy/orchestrated_IM'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
]

name_experiments = ['IM']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
IM,0.28 (0.004),95.039 (1.061),7.543 (0.032),89.6 (1.001),110.82 (1.728)


### Confidential_1000

In [8]:
PATH_TEST_LOG_MAS = '../simulated_data/Confidential_1000/main_results/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/Confidential_1000/main_results'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
]

name_experiments = ['AgentSim']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AgentSim,0.264 (0.009),143.402 (2.97),1.835 (0.109),14.159 (2.767),25.105 (3.442)


In [30]:
PATH_TEST_LOG_MAS = '../simulated_data/BPIC_2012_W/orchestrated_IM/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/BPIC_2012_W/orchestrated_IM'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
]

name_experiments = ['AgentSim']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AgentSim,0.256 (0.01),141.105 (15.43),2.09 (0.063),93.301 (12.73),92.72 (4.721)


In [31]:
PATH_TEST_LOG_MAS = '../simulated_data/BPIC_2017_W/autonomous_IM/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/BPIC_2017_W/autonomous_IM'

log_paths = [
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
]

name_experiments = ['AgentSim']

mean_results, results_df = main_(log_paths, name_experiments)
results_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AgentSim,0.353 (0.003),171.904 (3.449),2.4 (0.024),44.755 (2.568),56.756 (2.668)
